<a href="https://colab.research.google.com/github/madhumithapr/PIman/blob/master/remove_background.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import numpy as np
import os

In [0]:
def detect_edge(image, sigma=0.33):
    v = np.median(image)
    
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
    return edged

In [0]:
def image_resize(img):
    height, width = img.shape[:2]
    max_height = 600
    max_width = 600

# only shrink if img is bigger than required
    if max_height < height or max_width < width:
    # get scaling factor
        scaling_factor = max_height / float(height)
        if max_width/float(width) < scaling_factor:
            scaling_factor = max_width / float(width)
    # resize image
        img = cv2.resize(img, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)

    return img

In [0]:
def remove_bg(image, d_folder):
    BLUR = 21
    MASK_DILATE_ITER = 10
    MASK_ERODE_ITER = 10

#-- Read image -----------------------------------------------------------------------
    img = cv2.imread(image)
    img = image_resize(img)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#-- Edge detection -------------------------------------------------------------------
    edges = detect_edge(gray)
       
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)

    contour_info = []
    contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    for c in contours:
        contour_info.append((
            c,
            cv2.isContourConvex(c),
            cv2.contourArea(c),
        ))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    max_contour = contour_info[0]
    
    #-- Create empty mask, draw edges on it corresponding to largest contour ----
    # Mask is black, edge shape is white
    mask = np.zeros(edges.shape, dtype="uint8")
    cv2.fillConvexPoly(mask, max_contour[0], (255))

#-- Smooth mask, then blur it --------------------------------------------------------
    mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
    mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
    #mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)

    img_a = cv2.bitwise_and(img, img, mask=mask)
    
# save to disk
    img_name = image.split('/')[-1]
    dest = os.path.join(d_folder, 'masked_' + img_name)
    cv2.imwrite(dest, img_a)

In [0]:
input_folder = #Path

output_folder = #Path

for image_path in glob.glob(input_folder):
  remove_bg(image_path, output_folder)